# 🛠️ Telco Customer Churn - Feature Engineering

This notebook performs comprehensive feature engineering for the Telco Customer Churn dataset.

**Objectives**:
- Clean and preprocess data
- Create new meaningful features
- Encode categorical variables
- Scale numerical features
- Prepare data for machine learning

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)

print("🛠️ Feature Engineering Libraries Loaded!")

## 📥 Load Data

In [ ]:
# Load the dataset
try:
    df = pd.read_csv('../data/Telco-Customer-Churn.csv')
    print(f"✅ Dataset loaded: {df.shape}")
except FileNotFoundError:
    print("❌ Dataset not found. Please run 01_data_exploration.ipynb first")
    df = None

if df is not None:
    print("\n📊 Dataset overview:")
    print(df.head())

## 🧹 Data Cleaning

In [ ]:
# Data cleaning and preprocessing
print("🧹 DATA CLEANING")

# Check for data types and potential issues
print("\n📊 Current data types:")
print(df.dtypes)

# Check for any columns that should be numeric but are object type
print("\n🔍 Checking for numeric columns stored as objects:")
for col in df.columns:
    if df[col].dtype == 'object' and col not in ['customerID', 'Churn']:
        # Try to convert to numeric
        try:
            numeric_conversion = pd.to_numeric(df[col], errors='coerce')
            if numeric_conversion.notna().sum() > 0:
                print(f"  {col}: Can be converted to numeric")
                # Check for spaces or special characters
                unique_values = df[col].unique()
                print(f"    Unique values: {unique_values[:5]}")
        except:
            pass

In [ ]:
# Handle TotalCharges column (commonly has issues in this dataset)
if 'TotalCharges' in df.columns:
    print("🔧 Fixing TotalCharges column:")
    
    # Check current state
    print(f"Current dtype: {df['TotalCharges'].dtype}")
    print(f"Sample values: {df['TotalCharges'].head().tolist()}")
    
    # Convert to numeric, handling spaces and errors
    df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan)
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
    
    # Check for missing values
    missing_count = df['TotalCharges'].isna().sum()
    print(f"Missing values after conversion: {missing_count}")
    
    if missing_count > 0:
        # Fill missing values with median or 0 for new customers
        # If tenure is 0, set TotalCharges to 0
        if 'tenure' in df.columns:
            df.loc[(df['tenure'] == 0) & (df['TotalCharges'].isna()), 'TotalCharges'] = 0
            # Fill remaining missing values with median
            df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)
        else:
            df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)
        
        print(f"✅ Missing values handled")
    
    print(f"Final dtype: {df['TotalCharges'].dtype}")
    print(f"Final stats: {df['TotalCharges'].describe()}")

## 🎯 Target Variable Preparation

In [ ]:
# Prepare target variable
print("🎯 TARGET VARIABLE PREPARATION")

if 'Churn' in df.columns:
    # Convert target to binary
    df['Churn_Binary'] = (df['Churn'] == 'Yes').astype(int)
    
    print(f"Target distribution:")
    print(df['Churn'].value_counts())
    print(f"\nBinary target distribution:")
    print(df['Churn_Binary'].value_counts())
    print(f"\nChurn rate: {df['Churn_Binary'].mean():.2%}")
else:
    print("❌ Churn column not found")

## 🆕 Feature Creation

In [ ]:
# Create new features
print("🆕 CREATING NEW FEATURES")

# 1. Monthly charges per service ratio
if 'MonthlyCharges' in df.columns and 'tenure' in df.columns:
    # Average monthly charges (handling division by zero)
    df['AvgMonthlyCharges'] = df['MonthlyCharges']
    
    # Total charges per tenure month
    if 'TotalCharges' in df.columns:
        df['ChargesPerTenure'] = df['TotalCharges'] / (df['tenure'] + 1)  # +1 to avoid division by zero
    
    print("✅ Charges-based features created")

# 2. Service count features
service_cols = []
for col in df.columns:
    if any(service in col.lower() for service in ['phone', 'internet', 'online', 'backup', 'protection', 'support', 'streaming']):
        if df[col].dtype == 'object' and 'Yes' in df[col].unique():
            service_cols.append(col)

if service_cols:
    print(f"\nService columns identified: {service_cols}")
    
    # Count total services
    df['TotalServices'] = 0
    for col in service_cols:
        df['TotalServices'] += (df[col] == 'Yes').astype(int)
    
    # Service penetration rate
    df['ServicePenetration'] = df['TotalServices'] / len(service_cols)
    
    print(f"✅ Service count features created")
    print(f"Total services stats: {df['TotalServices'].describe()}")

# 3. Customer lifetime value features
if 'MonthlyCharges' in df.columns and 'tenure' in df.columns:
    # Customer lifetime value estimation
    df['EstimatedCLV'] = df['MonthlyCharges'] * df['tenure']
    
    # Monthly charges to tenure ratio
    df['ChargesTenureRatio'] = df['MonthlyCharges'] / (df['tenure'] + 1)
    
    print("✅ Customer value features created")

# 4. Contract and payment features
if 'Contract' in df.columns:
    # Contract length mapping
    contract_mapping = {
        'Month-to-month': 1,
        'One year': 12,
        'Two year': 24
    }
    df['ContractLength'] = df['Contract'].map(contract_mapping)
    
    # Long-term contract indicator
    df['LongTermContract'] = (df['ContractLength'] >= 12).astype(int)
    
    print("✅ Contract features created")

# 5. Demographic features
if 'SeniorCitizen' in df.columns and 'tenure' in df.columns:
    # Senior citizen with long tenure
    df['SeniorLongTenure'] = ((df['SeniorCitizen'] == 1) & (df['tenure'] > 12)).astype(int)
    
    print("✅ Demographic features created")

print(f"\n📊 Total features after creation: {df.shape[1]}")

## 🏷️ Categorical Encoding

In [ ]:
# Categorical variable encoding
print("🏷️ CATEGORICAL ENCODING")

# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Remove target and ID columns
exclude_cols = ['customerID', 'Churn']
categorical_cols = [col for col in categorical_cols if col not in exclude_cols]

print(f"Categorical columns to encode: {categorical_cols}")

# Create a copy for encoding
df_encoded = df.copy()

# Binary encoding for Yes/No columns
binary_cols = []
for col in categorical_cols:
    unique_values = df[col].unique()
    if len(unique_values) == 2 and set(unique_values) <= {'Yes', 'No'}:
        binary_cols.append(col)
        df_encoded[col] = (df[col] == 'Yes').astype(int)

print(f"Binary encoded columns: {binary_cols}")

# One-hot encoding for multi-category columns
remaining_categorical = [col for col in categorical_cols if col not in binary_cols]

if remaining_categorical:
    print(f"One-hot encoding columns: {remaining_categorical}")
    
    for col in remaining_categorical:
        # Create dummy variables
        dummies = pd.get_dummies(df[col], prefix=col, drop_first=True)
        df_encoded = pd.concat([df_encoded, dummies], axis=1)
        # Drop original column
        df_encoded.drop(col, axis=1, inplace=True)
    
    print("✅ One-hot encoding completed")

print(f"\n📊 Features after encoding: {df_encoded.shape[1]}")
print(f"New columns: {[col for col in df_encoded.columns if col not in df.columns]}")

## 📏 Feature Scaling

In [ ]:
# Feature scaling
print("📏 FEATURE SCALING")

# Identify numerical columns for scaling
exclude_from_scaling = ['customerID', 'Churn', 'Churn_Binary'] + \
                      [col for col in df_encoded.columns if col.endswith('_Yes') or col.endswith('_No')] + \
                      binary_cols

numerical_cols = df_encoded.select_dtypes(include=[np.number]).columns.tolist()
cols_to_scale = [col for col in numerical_cols if col not in exclude_from_scaling]

print(f"Columns to scale: {cols_to_scale}")

if cols_to_scale:
    # Create scaled version
    df_scaled = df_encoded.copy()
    
    # StandardScaler for features with normal distribution
    scaler = StandardScaler()
    df_scaled[cols_to_scale] = scaler.fit_transform(df_encoded[cols_to_scale])
    
    print("✅ Standard scaling completed")
    
    # Show scaling effect
    print("\n📊 Scaling effect:")
    scaling_comparison = pd.DataFrame({
        'Original_Mean': df_encoded[cols_to_scale].mean(),
        'Original_Std': df_encoded[cols_to_scale].std(),
        'Scaled_Mean': df_scaled[cols_to_scale].mean(),
        'Scaled_Std': df_scaled[cols_to_scale].std()
    })
    print(scaling_comparison.head())
else:
    df_scaled = df_encoded.copy()
    print("No columns need scaling")

## 🎯 Final Feature Selection

In [ ]:
# Final feature selection and preparation
print("🎯 FINAL FEATURE SELECTION")

# Remove unnecessary columns
columns_to_remove = ['customerID', 'Churn']  # Keep Churn_Binary as target
feature_columns = [col for col in df_scaled.columns if col not in columns_to_remove]

# Prepare final dataset
df_final = df_scaled[feature_columns].copy()

print(f"Final dataset shape: {df_final.shape}")
print(f"Features: {len(feature_columns) - 1}")  # -1 for target column
print(f"Target column: Churn_Binary")

# Show final feature list
features = [col for col in feature_columns if col != 'Churn_Binary']
print(f"\n📋 Final features ({len(features)}):")
for i, feature in enumerate(features, 1):
    print(f"{i:2d}. {feature}")

## 📊 Feature Correlation Analysis

In [ ]:
# Correlation analysis with target
print("📊 FEATURE CORRELATION WITH TARGET")

if 'Churn_Binary' in df_final.columns:
    # Calculate correlation with target
    target_correlation = df_final.corr()['Churn_Binary'].drop('Churn_Binary')
    target_correlation = target_correlation.sort_values(key=abs, ascending=False)
    
    print("Top 15 features correlated with churn:")
    print(target_correlation.head(15))
    
    # Visualize top correlations
    plt.figure(figsize=(10, 8))
    top_correlations = target_correlation.head(15)
    colors = ['red' if x < 0 else 'blue' for x in top_correlations.values]
    
    plt.barh(range(len(top_correlations)), top_correlations.values, color=colors, alpha=0.7)
    plt.yticks(range(len(top_correlations)), top_correlations.index)
    plt.xlabel('Correlation with Churn')
    plt.title('Top 15 Features Correlation with Churn')
    plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
    plt.tight_layout()
    plt.show()
    
    # Feature correlation matrix (top features)
    top_features = target_correlation.head(10).index.tolist() + ['Churn_Binary']
    correlation_matrix = df_final[top_features].corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                square=True, fmt='.2f')
    plt.title('Correlation Matrix - Top Features')
    plt.tight_layout()
    plt.show()

## ✂️ Remove Highly Correlated Features

In [ ]:
# Remove highly correlated features to reduce multicollinearity
print("✂️ REMOVING HIGHLY CORRELATED FEATURES")

# Calculate correlation matrix (excluding target)
feature_cols = [col for col in df_final.columns if col != 'Churn_Binary']
feature_corr_matrix = df_final[feature_cols].corr()

# Find highly correlated pairs
high_corr_pairs = []
threshold = 0.8  # Correlation threshold

for i in range(len(feature_corr_matrix.columns)):
    for j in range(i+1, len(feature_corr_matrix.columns)):
        if abs(feature_corr_matrix.iloc[i, j]) > threshold:
            col1 = feature_corr_matrix.columns[i]
            col2 = feature_corr_matrix.columns[j]
            corr_value = feature_corr_matrix.iloc[i, j]
            high_corr_pairs.append((col1, col2, corr_value))

if high_corr_pairs:
    print(f"Found {len(high_corr_pairs)} highly correlated pairs (threshold: {threshold}):")
    for col1, col2, corr in high_corr_pairs:
        print(f"  {col1} <-> {col2}: {corr:.3f}")
    
    # Remove features with lower correlation to target
    features_to_remove = set()
    for col1, col2, corr in high_corr_pairs:
        # Keep the feature with higher absolute correlation to target
        corr1 = abs(target_correlation.get(col1, 0))
        corr2 = abs(target_correlation.get(col2, 0))
        
        if corr1 < corr2:
            features_to_remove.add(col1)
        else:
            features_to_remove.add(col2)
    
    print(f"\nRemoving {len(features_to_remove)} features: {list(features_to_remove)}")
    
    # Create final dataset without highly correlated features
    final_features = [col for col in df_final.columns if col not in features_to_remove]
    df_final = df_final[final_features]
    
    print(f"Final dataset shape after correlation removal: {df_final.shape}")
else:
    print(f"No highly correlated features found (threshold: {threshold})")

print(f"\n📊 Final dataset ready for modeling: {df_final.shape}")

## 💾 Save Processed Data

In [ ]:
# Save the final processed dataset
print("💾 SAVING PROCESSED DATA")

# Save full processed dataset
df_final.to_csv('../data/telco_processed_features.csv', index=False)
print("✅ Full processed dataset saved as '../data/telco_processed_features.csv'")

# Save feature names for future use
feature_names = [col for col in df_final.columns if col != 'Churn_Binary']
pd.DataFrame({'feature_names': feature_names}).to_csv('../data/feature_names.csv', index=False)
print("✅ Feature names saved as '../data/feature_names.csv'")

# Save preprocessing summary
preprocessing_summary = {
    'original_shape': df.shape,
    'final_shape': df_final.shape,
    'features_created': df_final.shape[1] - df.shape[1],
    'churn_rate': df_final['Churn_Binary'].mean(),
    'total_features': len(feature_names),
    'categorical_encoded': len(binary_cols) + len(remaining_categorical),
    'features_scaled': len(cols_to_scale) if 'cols_to_scale' in locals() else 0
}

summary_df = pd.DataFrame([preprocessing_summary])
summary_df.to_csv('../data/preprocessing_summary.csv', index=False)
print("✅ Preprocessing summary saved as '../data/preprocessing_summary.csv'")

print("\n📋 FEATURE ENGINEERING SUMMARY:")
print(f"✅ Original dataset: {df.shape}")
print(f"✅ Final dataset: {df_final.shape}")
print(f"✅ Features for modeling: {len(feature_names)}")
print(f"✅ Churn rate: {df_final['Churn_Binary'].mean():.2%}")
print(f"✅ Data ready for machine learning!")

print("\n🚀 Next steps:")
print("1. Run notebook: 03_exploratory_data_analysis.ipynb")
print("2. Run notebook: 04_machine_learning_models.ipynb")
print("3. Run notebook: 05_model_evaluation.ipynb")